In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy

In [ ]:
import mediapy as media
import random
import sys
import torch

from diffusers import AutoPipelineForText2Image

pipe = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/sdxl-turbo",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    )

pipe = pipe.to("cuda")

In [ ]:
def remove_blank_lines(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            if line.strip():  # Check if the line is not just whitespace
                outfile.write(line)

# Example usage:
input_file_path = '/content/school ahead.txt'
output_file_path = '/content/pedestrain1 f.txt'

remove_blank_lines(input_file_path, output_file_path)

In [ ]:
import os

In [ ]:
# Assuming you have a function 'save_image' to save the generated image
def save_image(image, output_path):
    image.save(output_path)

input_file = "/content/final prompts.txt"  # Replace with the path to your input file
output_folder = "/content/finalimages"   # Replace with the desired output folder

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

num_inference_steps = 4
seed = random.randint(0, sys.maxsize)

# Read prompts from the input file
with open(input_file, 'r') as file:
    prompts = file.readlines()

# Iterate through each prompt
for i, prompt in enumerate(prompts):
    prompt = prompt.strip()

    # Generate images for the current prompt
    images = pipe(
        prompt=prompt,
        guidance_scale=0.8,  # Adjust the guidance scale as needed
        viewpoint_guidance="outside",  # Add a parameter for viewpoint guidance
        num_inference_steps=num_inference_steps,
        generator=torch.Generator("cuda").manual_seed(seed),
    ).images

    # Save the generated images in the output folder
    for j, image in enumerate(images):
        output_path = os.path.join(output_folder, f"{i + 1}.jpg")
        save_image(image, output_path)

    print(f"Generated images for prompt {i + 1}")

print("All images generated and saved.")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/sample_data/140images


In [ ]:
!zip -r newped1 /content/pedestrain_new1

In [ ]:
from google.colab import files
files.download('/content/schoolahead1.zip')

In [ ]:
import re

def replace_t_with_y(text):
    return re.sub(r'T', 'Y', text)

def main():
    # Input file path
    file_path = input("/content/T junction new.txt")

    try:
        # Read the content of the file
        with open(file_path, 'r') as file:
            original_text = file.read()

        # Replace 'T' with 'Y'
        modified_text = replace_t_with_y(original_text)

        # Output the modified text
        print("Original text:")
        print(original_text)
        print("\nModified text:")
        print(modified_text)

    except FileNotFoundError:
        print("File not found. Please check the file path.")

if __name__ == "__main__":
    main()


In [ ]:
import torch
from diffusers.utils import load_image
from diffusers import StableDiffusionXLControlNetInpaintPipeline, ControlNetModel
from transformers import pipeline
from PIL import Image
import numpy as np

# Configuration
IS_UNDER_EXPOSURE = False #change this option for output underexposured ball
if IS_UNDER_EXPOSURE:
    PROMPT = "a perfect Red car going through the hills with clear sky"
else:
    PROMPT = "a perfect Red car going through the hills with clear sky"

NEGATIVE_PROMPT = "matte, diffuse, flat, dull"
IMAGE_URL = "/content/download.jpeg"

# load pipeline
controlnet = ControlNetModel.from_pretrained("diffusers/controlnet-depth-sdxl-1.0", torch_dtype=torch.float16)
pipe = StableDiffusionXLControlNetInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    torch_dtype=torch.float16,
).to("cuda")
pipe.load_lora_weights("DiffusionLight/DiffusionLight")
pipe.fuse_lora(lora_scale=0.75)
depth_estimator = pipeline(task="depth-estimation", model="Intel/dpt-large")

# prepare input image
init_image = load_image(IMAGE_URL)
depth_image = depth_estimator(images=init_image)['depth']

# create mask and depth map with mask for inpainting
def get_circle_mask(size=256):
    x = torch.linspace(-1, 1, size)
    y = torch.linspace(1, -1, size)
    y, x = torch.meshgrid(y, x)
    z = (1 - x**2 - y**2)
    mask = z >= 0
    return mask
mask = get_circle_mask().numpy()
depth = np.asarray(depth_image).copy()
depth[384:640, 384:640] = depth[384:640, 384:640] * (1 - mask) + (mask * 255)
depth_mask = Image.fromarray(depth)
mask_image = np.zeros_like(depth)
mask_image[384:640, 384:640] = mask * 255
mask_image = Image.fromarray(mask_image)

# run the pipeline
output = pipe(
    prompt=PROMPT,
    negative_prompt=NEGATIVE_PROMPT,
    num_inference_steps=30,
    image=init_image,
    mask_image=mask_image,
    control_image=depth_mask,
    controlnet_conditioning_scale=0.5,
)

# save output
output["images"][0].save("output.png")


In [ ]:
!pip install diffusers